In [1]:
import logging, os

env = "test"
log_file = os.path.normpath("C:/Temp/bcu/log/get-persons-{}.log".format(env))
file_mode = 'w'
logging.basicConfig(filename=log_file, filemode=file_mode, level=logging.INFO, format='%(asctime)s %(levelname)s : %(message)s')

In [2]:
def get_associations(affiliations, une_id, staff, ass_type):
    for une_id in une_ids:
        for aff in staff:
            assoc = dict()
            assoc['une_id'] = une_id
            assoc['org_id'] = aff['organization']['uuid']
            assoc['type'] = ass_type
            if aff.get('period'):
                assoc['start_date'] = aff.get('period').get('startDate')
                assoc['end_date'] = aff.get('period').get('end_date')
            assoc['primary'] = aff['primaryAssociation']
            affiliations.append(assoc)

In [3]:
import requests, json, psycopg2

test_key = os.path.normpath('C:/Users/MohapatraD/.secrets/BCU/pure-apikey-staging.txt')
#prod_key = os.path.normpath('C:/Users/MohapatraD/.secrets/BCU/pure-apikey-prod.txt')
test_url = "https://cristest.bcu.ac.uk/ws/api"
#prod_url = "https://pure.bcu.ac.uk/ws/api"


if env == "prod":
    url = prod_url
    key= prod_key
else:
    url = test_url
    key= test_key

logging.info("Processing data in {}".format(env))

with open(key, 'r') as keyfile:
    api_key = keyfile.read().rstrip()
    

headers = {'api-key': api_key}

PAGE_SIZE  = 150
START_FROM = 0

# Iterate through all items to collect UUIDs

processed = 0
offset = START_FROM
conn = psycopg2.connect(host="localhost", port=5432, database="BCU", user="postgres", password="admin")
sql = "insert into persons(id, body) values (%s, %s::jsonb) on conflict(id) do update set body = excluded.body"
cur = conn.cursor()
loop = True
while loop:
    params = {'offset': offset, 'size': PAGE_SIZE}
    logging.info('Collecting persons {} - {} '.format(offset, (offset + PAGE_SIZE - 1)))
    resp = requests.get('{}/persons'.format(url), headers = headers, params = params)
    resp.raise_for_status()
    content = resp.json()
    count = content['count']
    logging.info('Total number of persons: {}'.format(count))
    items = content['items']
    logging.info('Persons in this response: {}'.format(len(items)))
    for item in items:
        id = item['pureId']
        cur.execute(sql, (id, json.dumps(item)))
        conn.commit()
    offset += PAGE_SIZE
    if offset > count:
        loop = False
cur.close()
conn.close()
logging.info('All persons collected')